# 什么是生成建模？

1. 生成建模的含义
2. 与判别模型的区别
3. 生成建模框架
4. 构造生成建模的通用方法

生成模型从概率角度描述了生成数据集的方法。从该模型中抽样即可生成新数据。  

> *Generative modeling is a branch of machine learning that involves training a model to produce new data that is similar to a given dataset.*  生成建模是一种机器学习算法，使用给定数据生成新的、类似的新数据。


## 生成建模的过程
生成模型从某一类训练数据（**观测**）中习得**特征/规则**，加入**随机噪声**后生成新的样本。换句话说，观测可以由这一套规则反推得到。
>- 生成模型必须是**probabilistic**，不能是**deterministic**，否则模型每次都只会有一个固定输出；
>- 模型必须包括一个**随机元素stochastic**，以影响模型生成的每个样本。

##  生成建模的目的
建立一个能够尽可能模拟真实训练集中存在的概率分布，从中抽样，生成独特的新观测，就好像原始训练数据集中就包括它们。

## 与**判别模型discriminative modeling**的区别
>- 判别建模时训练集中的每一个观测都有一个标签（监督学习）；给定观测值x，标签y出现的概率；为了高置信度的判断
>- 生成建模通常使用没有标签的数据集（无监督学习）；观测值x出现的概率；为了高质量的生成

## 生成建模的意义
1. 高质量的数据分类技术虽然有用，单全面了解数据最初的生成方式才是关键
2. 生成建模能够助力强化学习克服复杂高昂的数值模拟
3. 人脑==生成建模

## 生成建模框架
>- 已知观测数据x
>- 假设观测是由某一未知规则/分布 $P_{\text{data}}(x)$ 生成的
>- 生成模型P_model试图拟合 $P_{\text{data}}(x)$
>- 如果拟合成功，则可以从P_model中采样并生成观测，这些样本看上去就好像是从 $P_{\text{data}}(x)$ 中直接提取的<br>

> $P_{\text{model}}(x)$ 建模成功
>> Rule 1. 模型生成的采样看上去就好像是从 $P_{\text{data}}(x)$ 中提取的<br>
>> Rule 2. 模型生成的采样与X中的观测明显不同，换句话说，模型不应该简单地复制已有的数据点

# 概率生成模型

## 四个关键术语

### 样本空间
是观测 X 可以获取的所有值的完整集合。

---


### 概率密度函数 $P(x)$

概率密度函数是描述数据在样本空间上如何分布的**规则**。可以将样本空间中的点 $x$ 映射成 $0~1$ 的数字，样本空间中所有点的密度函数值和必须等于1。

- **真实分布 $P_{\text{data}}(x)$**
    
    一个未知的、神圣的分布，它真正决定了数据如何产生。我们的终极目标就是尽可能地了解它。

- **模型分布 $P_{\text{model}}(x; \theta)$**
    
    这是我们设计的、用来逼近/拟合 $P_{\text{data}}(x)$ 的一个可计算的分布，其中，$\theta$ 代表模型的参数。

    潜在的模型分布 $P_{\text{model}}(x; \theta)$ 有无限种可能的选择，我们使用**参数化建模**系统地寻找这些参数。

---


### 参数化建模
我们不是漫无目的地搜索所有可能的函数，而是选择一个由有限维参数 $\theta$ 控制的**模型族** $P_{\theta}(x)$。
- 在世界地图的例子中，"所有可能绘制的矩形"构成了一个模型族。
- 每一个具体的矩形（即一个具体的 $P_{\theta}(x)$）由四个参数唯一确定：
  $\theta = (\theta_1, \theta_2, \theta_3, \theta_4)$
  这四个参数分别代表了矩形左下角和右上角的坐标。


- 这里的**模型族**在深度学习中指的是个特定的神经网络结构（例如，VAE、GAN 或 Diffusion Model）。
- 参数 $\theta$ 就是该神经网络中所有的**权重**和**偏置**。

- 参数化是将一个在**无限函数空间**中的搜索问题，转化为了一个在**有限维参数空间**中的优化问题。

---


### 似然

现在我们确定了模型族 $P_{\theta}(x)$ 和一组观测到的数据 $X$。我们使用**似然**来衡量某一组具体参数 $\theta$ 的好坏。

似然函数 $\mathcal{L}(\theta | X)$ 定义为给定参数 $\theta$ 时，观察到整个数据集 $X$ 的概率：
$\mathcal{L}(\theta | X) = \prod_{x \in X} P_{\theta}(x)$

换句话说，在假设数据是*由分布 $P_{\theta}$ 产生的前提下，观察到我们手头这批数据 $X$ 的可能性有多大*。

- 似然关注的是哪个参数 $\theta$ 更能"解释"已有的数据，而不是数据本身的概率。

- $\mathcal{L}(\theta | X)$ **不是** $\theta$ 的概率分布，因此其在参数空间上的积分不为 1。

- 出于数值计算稳定性考虑，我们几乎总是使用**对数似然**：
    $\ell(\theta | X) = \sum_{x \in X} \log P_{\theta}(x)$

    它将连乘变成了求和，有效防止数值下溢并简化求导运算。

---
### 最大似然估计

参数化建模的最终目标是找到最合理的 $\hat{\theta}$ ，以最大化数据集 $X$ 观测值的似然性。

最大似然估计 $\hat{\theta}_{\text{MLE}}$ 被定义为：

$
\hat{\theta}_{\text{MLE}} = \arg \max_{\theta} \mathcal{L}(\theta | X) = \arg \max_{\theta} \ell(\theta | X)
$

其中：
- $\mathcal{L}(\theta | X)$ 是似然函数
- $\ell(\theta | X)$ 是对数似然函数


由于神经网络框架通常被设计为**最小化**损失函数，我们通常将其转化为最小化**负对数似然**：
$
\hat{\theta}_{\text{MLE}} = \arg \min_{\theta} (-\ell(\theta | X))
$

**当我们训练一个生成式模型时，实际上是在寻找能够最大化训练数据似然（或等价地，最小化负对数似然）的神经网络参数**

---

# 表示学习
- 表示学习就是让模型自动从高维数据中学习出它的本质、低维、有意义的结构。
- 表示学习能学习对于描述给定的观测最为重要的特征，以及如何根据原始数据生成这些特征。
- 从数学角度来讲，它会设法找出数据表现出的高度非线性流形，然后建立充分描述该空间所需的维度。

1. 从高维（像素） →  低维（潜在变量）

    例如一张图像有上千个像素维度，但实际控制图像内容的因素很少：

>    物体类别（狗/车/苹果）、形状（瘦/胖、高/低）、姿态、颜色、光照、背景

    这些都可以在一个 潜在空间 z 中用几十或几百维来表示。

2. 低维表示是可解释、可操作的

    在 latent space 中：

>    改变某个维度 = 调整某个高层属性（如让罐子变高、让狗变成金毛、让脸变年轻）

    在 latent space 中插值 = 做风格迁移、生成新图像

3. 模型通过 encoder/decoder 自动学习这种映射x→z→x

>    Encoder 把复杂输入压缩到潜在空间

>    Decoder 从潜在表示重新生成数据

    这是 VAE、GAN、Diffusion 模型的共同结构。

4. **生成模型的本质就是理解数据的低维结构，生成模型全部依赖 latent space 的结构**

- 在 latent space 中进行采样、操作，再 decode 出新样本
- 没有良好的表示学习，就无法生成新图/做风格迁移/做隐变量推断/做多样性控制/做数据重建

